In [26]:
# #############################################################################
# Importing necessary libraries for creating model

from keras.models import model_from_json
from keras.models import load_model
import os
import pandas as pd
import numpy as np
import keras
import cv2 # importing open CV library
import imutils # Package of convenience functions
import dlib   # For frontal face identity
from PIL import Image
from skimage import io
import matplotlib.pyplot as plt
import csv
import requests
import face_recognition # importing face recognition library
from sklearn.metrics import classification_report, confusion_matrix
import timeit
import sys




In [55]:

# #############################################################################
# loading the trained model and weights
json_file = open('../trained model/model.json', 'r')     # Loading the jason file
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("../trained model/model.h5")    # Loading the weights


def detect_faces(image):                    # Create a face detector
    face_detector = dlib.get_frontal_face_detector()
    detected_faces = face_detector(image, 1)    # Run detector and get bounding boxes of the faces on image
    face_frames = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]  # To identify the frame size
    return face_frames


def resize_im(image, hight, width):          # Resizing the image
    resized_im = image.resize((hight, width))
    return resized_im


def reshape(numpy_array):            # Reshape the image
    d = np.stack(numpy_array)
    reshaped = d.reshape(1, d.size)
    return reshaped


def GIGIDL(image_path):
    f = open("../storage.csv", "w")
    img= cv2.imread(image_path)
    detected_faces = detect_faces(img)
    for n, face_rect in enumerate(detected_faces):
                    fa = Image.fromarray(img).crop(face_rect)   # Cropping the face or faces
                    s = np.array(resize_im(fa, 100,100))
                    c = cv2.cvtColor(s, cv2.COLOR_BGR2GRAY)    # Converting the image
                    re_data = reshape(c)
                    np.savetxt(f, re_data, delimiter=",", fmt = '%d')  # Save the converted image data
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()    # Need to kill the image window
    #reading the faces from the temp file
    data = pd.read_csv('../storage.csv', header = None)
    df_x = data.iloc[:,0:].values.reshape(len(data),100,100,1)   # Creating the requird formate
    f_x = np.array(df_x)  # Need to convert as a array
    loaded_model.compile(loss='categorical_crossentropy',optimizer = 'adadelta', metrics = ['accuracy'])
    prob = np.asmatrix(loaded_model.predict(f_x))
    if prob[0,0] > prob[0,1]:
             gender= 'F'
    if prob[0,0] < prob[0,1]:
             gender='M'
    return gender
    



In [76]:
#variables
file_path= '/Users/nadabeili/Desktop/Improved-Gender-Identifier/male/OUI_African/'
out_path ='/Users/nadabeili/Desktop/Improved-Gender-Identifier/'
result_file= out_path + 'African_male.csv'

In [ ]:
with open(result_file, 'w', newline='') as myfile:             # Here output written in the file
    wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
    wr.writerow(['Image_name','True_Gender','Pred_Gender'])

    for imageName in os.listdir(file_path):                        # Get the images from directory
        if imageName[-3:]in ['jpeg','png','jpg']: #check if it is image or not
            try:
                image_location=file_path+imageName                     # Get the image loacation of each image
                GIGIDL_gender=GIGIDL(image_location)   # return the predicted gender
                wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
                wr.writerow([imageName,'M',GIGIDL_gender])
            except:
                wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
                wr.writerow([imageName,'M','U'])   #if there is an error in returning gender , write undefined
                print("Oops!",sys.exc_info()[0],"occured.")
                print()

Oops! <class 'pandas.errors.EmptyDataError'> occured.

Oops! <class 'pandas.errors.EmptyDataError'> occured.

Oops! <class 'pandas.errors.EmptyDataError'> occured.

Oops! <class 'pandas.errors.EmptyDataError'> occured.



In [ ]:
### Precision, recall, F1_score
#read the result_file
male = pd.read_csv(out_path + 'African_male.csv')
female = pd.read_csv(out_path + 'African_female.csv')
#Merge the 2 dataframes
results_data = pd.concat([female, male])
#find the number of image
nbr_image= len(results_data)
print('Total number of images: ',nbr_image)
#remove undefined gender
results_data=results_data[results_data['Pred_Gender']!='U']
print('Number of images with predicted gender: ',len(results_data),'\n')

# True values
y_true = results_data['True_Gender']
# Predicted values
y_pred = results_data['Pred_Gender']

# Print the confusion matrix
print(confusion_matrix(y_true, y_pred))
# Print the precision and recall, among other metrics
print(classification_report(y_true, y_pred, digits=4))

#printing the metrics
metrics_dict=classification_report(y_true, y_pred,output_dict=True)

#precision:
print('Precision:',round(metrics_dict['weighted avg']['precision'],4))
#Recall
print('Recall:',round(metrics_dict['weighted avg']['recall'],4))
#F1-score
print('F1-score:',round(metrics_dict['weighted avg']['f1-score'],4))
#accuracy
print('Accuracy:',round(metrics_dict['accuracy'],4))